In [1]:
import pandas as pd
import mysql.connector as mc
import matplotlib
import calendar

In [2]:
db = mc.connect(
  host='172.18.55.6',
  user='comandato',
  passwd='comandato123',
)
cur= db.cursor()

In [3]:
Dia='16'
Mes='05'
Anio='2019'
fecha_pago=Dia+Mes+Anio

In [4]:
F_Mes='MAYO'

# OBTENER GESTION DE ASESOR

### CONVENIO

In [5]:
querry_gestion_cuotas="""SELECT NRO_IDENTIFICACION_CLIENTE,COD_AGENTE,CAST(DATE_FORMAT(FECHA_GESTION,'%d') as UNSIGNED) as FECHA_GESTION,CONTACTO_GESTIONO,ACCION_REALIZADA,RESPUESTA_OBTENIDA,NUMERO_GESTION FROM SISTEMECUADOR_ATM.GESTION where FECHA_GESTION BETWEEN '{0}-{1}-00 00:00:00' AND '{0}-{1}-{2} 21:00:00';""".format(Anio,Mes,Dia)
gestion_cuotas=pd.read_sql(querry_gestion_cuotas, con=db)

In [6]:
gestion_cuotas.head()

,NRO_IDENTIFICACION_CLIENTE,COD_AGENTE,FECHA_GESTION,CONTACTO_GESTIONO,ACCION_REALIZADA,RESPUESTA_OBTENIDA,NUMERO_GESTION
0,0104772363,D013113,2,NO CONTACTO,HACER LLAMADA,NO CONTESTA,0984386351
1,0917597551,D013110,2,NO CONTACTO,HACER LLAMADA,BUZON DE MENSAJES,0991138842
2,0916719172,D013113,2,TITULAR,HACER LLAMADA,VOLVER A LLAMAR,0979811719
3,0911637395,D013113,2,NO CONTACTO,HACER LLAMADA,NO CONTESTA,0999508037
4,0909622037,D013110,2,TITULAR,HACER LLAMADA,ACUERDO DE PAGO,0999113107


In [7]:
gestion_asesor_cuotas=gestion_cuotas[(gestion_cuotas['ACCION_REALIZADA']=='HACER LLAMADA') & ((gestion_cuotas['CONTACTO_GESTIONO']=='CLIENTE') | (gestion_cuotas['CONTACTO_GESTIONO']=='TITULAR') |(gestion_cuotas['CONTACTO_GESTIONO']=='TERC. VALIDO'))& (gestion_cuotas['RESPUESTA_OBTENIDA']!='YA PAGO')]

In [8]:
gestion_asesor_cuotas=gestion_asesor_cuotas.sort_values(by=['FECHA_GESTION'],ascending=[True])
gestion_asesor_cuotas=gestion_asesor_cuotas.drop_duplicates(['NRO_IDENTIFICACION_CLIENTE'], keep='first')
gestion_asesor_cuotas = gestion_asesor_cuotas.rename(columns={'NRO_IDENTIFICACION_CLIENTE':'IDENTIFICACION'})

### CORRIENTE

In [9]:
querry_gestion="""SELECT NRO_IDENTIFICACION_CLIENTE,COD_AGENTE,CAST(DATE_FORMAT(FECHA_GESTION,'%d') as UNSIGNED) as FECHA_GESTION,CONTACTO_GESTIONO,ACCION_REALIZADA,RESPUESTA_OBTENIDA,NUMERO_GESTION FROM SISTEMECUADOR_ATM.GESTION where FECHA_GESTION BETWEEN '{0}-{1}-00 00:00:00' AND '{0}-{1}-{2} 21:00:00' AND COMENTARIOS_GESTIONO NOT LIKE '%CAMPAÑA CONVENIO%';""".format(Anio,Mes,Dia)
gestion=pd.read_sql(querry_gestion, con=db)

In [10]:
gestion_asesor=gestion[(gestion['ACCION_REALIZADA']=='HACER LLAMADA') & ((gestion['CONTACTO_GESTIONO']=='CLIENTE') | (gestion['CONTACTO_GESTIONO']=='TITULAR') |(gestion['CONTACTO_GESTIONO']=='TERC. VALIDO'))& (gestion['RESPUESTA_OBTENIDA']!='YA PAGO')]

In [11]:
gestion_asesor=gestion_asesor.sort_values(by=['FECHA_GESTION'],ascending=[True])

In [12]:
gestion_asesor=gestion_asesor.drop_duplicates(['NRO_IDENTIFICACION_CLIENTE'], keep='first')

In [13]:
gestion_asesor.head()

,NRO_IDENTIFICACION_CLIENTE,COD_AGENTE,FECHA_GESTION,CONTACTO_GESTIONO,ACCION_REALIZADA,RESPUESTA_OBTENIDA,NUMERO_GESTION
2,0916719172,D013113,2,TITULAR,HACER LLAMADA,VOLVER A LLAMAR,0979811719
482,0923008759,D013116,2,TITULAR,HACER LLAMADA,INFRACTOR CUELGA LLAMADA,0980571447
481,1312954082,D013126,2,TITULAR,HACER LLAMADA,LOCALIZADO SIN ACUERDO,0996780491
479,0911723658,D013133,2,TITULAR,HACER LLAMADA,INFRACTOR CUELGA LLAMADA,0988307748
478,0913775326,D013116,2,TITULAR,HACER LLAMADA,ACUERDO DE PAGO,0939589900


# OBTENER GESTION IVR

In [14]:
querry_masivos="""SELECT NRO_IDENTIFICACION_CLIENTE,COD_AGENTE,CAST(DATE_FORMAT(FECHA_GESTION,'%d') as UNSIGNED) as FECHA_GESTION,CONTACTO_GESTIONO,ACCION_REALIZADA,RESPUESTA_OBTENIDA,NUMERO_GESTION FROM SISTEMECUADOR_ATM.gestion_ivr where FECHA_GESTION BETWEEN '{0}-{1}-00 00:00:00' AND '{0}-{1}-{2} 21:00:00';""".format(Anio,Mes,Dia)
gestion_masivos=pd.read_sql(querry_masivos, con=db)

In [15]:
gestion_ivr=gestion_masivos[((gestion_masivos['ACCION_REALIZADA']=='ENVIAR IVR') & (gestion_masivos['RESPUESTA_OBTENIDA']=='CONTESTA IVR'))|((gestion_masivos['ACCION_REALIZADA']=='ENVIAR SMS') & (gestion_masivos['RESPUESTA_OBTENIDA']=='SMS ENVIADO'))|((gestion_masivos['ACCION_REALIZADA']=='ENVIAR WHATSAPP') & (gestion_masivos['RESPUESTA_OBTENIDA']=='ENVIO WHATSAPP'))]

In [16]:
gestion_ivr.tail()

,NRO_IDENTIFICACION_CLIENTE,COD_AGENTE,FECHA_GESTION,CONTACTO_GESTIONO,ACCION_REALIZADA,RESPUESTA_OBTENIDA,NUMERO_GESTION
67647,0924932684,D05554,16,TITULAR,ENVIAR IVR,CONTESTA IVR,0996793546
67648,0928443639,D05554,16,TITULAR,ENVIAR IVR,CONTESTA IVR,0985184760
67649,0929222313,D05554,16,TITULAR,ENVIAR IVR,CONTESTA IVR,042378488
67650,0931070072,D05554,16,TITULAR,ENVIAR IVR,CONTESTA IVR,0996815947
67651,0940393309,D05554,16,TITULAR,ENVIAR IVR,CONTESTA IVR,0996340027


# OBTENER GESTION SMS

In [17]:
gestion_sms=gestion_masivos[gestion_masivos['ACCION_REALIZADA']=='ENVIAR SMS']

In [18]:
gestion_sms.head()

,NRO_IDENTIFICACION_CLIENTE,COD_AGENTE,FECHA_GESTION,CONTACTO_GESTIONO,ACCION_REALIZADA,RESPUESTA_OBTENIDA,NUMERO_GESTION
11170,0102381803,D05554,6,TITULAR,ENVIAR SMS,SMS ENVIADO,0992139760
11171,1102427356,D05554,6,TITULAR,ENVIAR SMS,SMS ENVIADO,0993404025
11172,1203176514,D05554,6,TITULAR,ENVIAR SMS,SMS ENVIADO,0993096912
11173,1205250648,D05554,6,TITULAR,ENVIAR SMS,SMS ENVIADO,0967399139
11174,1207073964,D05554,6,TITULAR,ENVIAR SMS,SMS ENVIADO,0996528201


In [19]:
gestiones=[gestion_asesor,gestion_ivr,gestion_sms]
total_gestiones=pd.concat(gestiones)

In [20]:
total_gestiones.head()

,NRO_IDENTIFICACION_CLIENTE,COD_AGENTE,FECHA_GESTION,CONTACTO_GESTIONO,ACCION_REALIZADA,RESPUESTA_OBTENIDA,NUMERO_GESTION
2,0916719172,D013113,2,TITULAR,HACER LLAMADA,VOLVER A LLAMAR,0979811719
482,0923008759,D013116,2,TITULAR,HACER LLAMADA,INFRACTOR CUELGA LLAMADA,0980571447
481,1312954082,D013126,2,TITULAR,HACER LLAMADA,LOCALIZADO SIN ACUERDO,0996780491
479,0911723658,D013133,2,TITULAR,HACER LLAMADA,INFRACTOR CUELGA LLAMADA,0988307748
478,0913775326,D013116,2,TITULAR,HACER LLAMADA,ACUERDO DE PAGO,0939589900


In [21]:
total_gestiones=total_gestiones.drop_duplicates(['NRO_IDENTIFICACION_CLIENTE'], keep='first')

In [22]:
total_gestiones = total_gestiones.rename(columns={'NRO_IDENTIFICACION_CLIENTE':'IDENTIFICACION'})

In [23]:
total_gestiones.head()

,IDENTIFICACION,COD_AGENTE,FECHA_GESTION,CONTACTO_GESTIONO,ACCION_REALIZADA,RESPUESTA_OBTENIDA,NUMERO_GESTION
2,0916719172,D013113,2,TITULAR,HACER LLAMADA,VOLVER A LLAMAR,0979811719
482,0923008759,D013116,2,TITULAR,HACER LLAMADA,INFRACTOR CUELGA LLAMADA,0980571447
481,1312954082,D013126,2,TITULAR,HACER LLAMADA,LOCALIZADO SIN ACUERDO,0996780491
479,0911723658,D013133,2,TITULAR,HACER LLAMADA,INFRACTOR CUELGA LLAMADA,0988307748
478,0913775326,D013116,2,TITULAR,HACER LLAMADA,ACUERDO DE PAGO,0939589900


# MULTAS ASIGNADAS

In [24]:
multas="""SELECT *   FROM SISTEMECUADOR_ATM.MULTAS_MENSUALES where Mes={0}""".format(Mes)
resultado_multas=pd.read_sql(multas, con=db)

In [25]:
resultado_multas.head()

,Cedula,No_multa,Anio,Mes,Dia,Dia_retiro,Dias_mora,Saldo,Cartera,Segmento,Descripcion,Placa,Pago,dia_pago,tipo
0,00042448K,5722394,2019,5,1,None,191.0,115.80,CORRIENTE,MULTAS NUEVAS,COND.EXCEDA DENTRO RANGO MODERADO LÍMITES DE V...,GRP0269,None,None,None
1,1000005262,4854542,2018,5,2,None,52.0,75.86,CONVENIO,CUOTA,Atraso,nan,None,None,None
2,1000041952,4087528,2018,5,2,None,232.0,112.50,CORRIENTE,FASE II,COND.EXCEDA DENTRO RANGO MODERADO LÍMITES DE V...,ABA1143,None,None,None
3,1000131977,1786073,2018,5,2,None,737.0,54.90,CORRIENTE,MENOR CUANTIA,DETEN.ESTAC.VEH.LUGAR NO PERMITIDO DEJAR/RECIB...,PAA6834,None,None,None
4,1000203719,5039556,2018,5,2,None,45.0,64.55,CONVENIO,CUOTA,Atraso,nan,None,None,None


In [26]:
dias_mora_multa=resultado_multas[['No_multa','Dias_mora']].copy()

In [27]:
dias_mora_multa = dias_mora_multa.rename(columns={'No_multa':'ID_FACTURA','Dias_mora':'DIAS_MORA'})
dias_mora_multa['ID_FACTURA']=dias_mora_multa['ID_FACTURA'].apply(int)

In [28]:
dias_mora_multa.head()

,ID_FACTURA,DIAS_MORA
0,5722394,191.0
1,4854542,52.0
2,4087528,232.0
3,1786073,737.0
4,5039556,45.0


# PAGOS

In [29]:
pago_corr={
    'ruta':'..\..\..\BASES DATATOOLS\PAGOS\\2019\\'+F_Mes+'\Sitema de cobro '+fecha_pago+'.xlsx',
    'hoja':'PAGOS_CORRIENTE',
    'cartera':'CORRIENTE',
    'identificador':'IDENTIFICACION'
}

pago_cuot={
    'ruta':'..\..\..\BASES DATATOOLS\PAGOS\\2019\\'+F_Mes+'\\Sitema de cobro '+fecha_pago+'.xlsx',
    'hoja':'PAGOS_CUOTACERO',
    'cartera':'CUOTA',
    'identificador':'ID'
}

pago_conv={
    'ruta':'..\..\..\BASES DATATOOLS\PAGOS\\2019\\'+F_Mes+'\\Sitema de cobro '+fecha_pago+'.xlsx',
    'hoja':'PAGOS_CONVENIOS',
    'cartera':'CONVENIO',
    'identificador':'ID'
}

In [30]:
pagos_corr=pd.read_excel(open(pago_corr['ruta'],'rb'), sheet_name=pago_corr['hoja'], dtype={pago_corr['identificador']: str})
pagos_cuot=pd.read_excel(open(pago_cuot['ruta'],'rb'), sheet_name=pago_cuot['hoja'], dtype={pago_cuot['identificador']: str})
pagos_conv=pd.read_excel(open(pago_conv['ruta'],'rb'), sheet_name=pago_conv['hoja'], dtype={pago_conv['identificador']: str})

In [31]:
def agregar_ceros(base):
    ced=base['ID']
    if len(ced)==9 or len(ced)==12 :
        if ced.isdigit():
            return '0'+ced
    return ced

In [32]:
pagos_cuot['ID']=pagos_cuot.apply(agregar_ceros,axis=1)

### CORRIENTE

In [33]:
def agregar_ceros2(base):
    ced=base['IDENTIFICACION']
    if len(ced)==9 or len(ced)==12 :
        if ced.isdigit():
            return '0'+ced
    return ced

In [34]:
pagos_corr['IDENTIFICACION']=pagos_corr.apply(agregar_ceros2,axis=1)

In [35]:
def tipo(base):
    casa=base['CASA_COBRO']
    if casa=='SISTEMA SALDO MENORES':
        return 'MENORES'
    else:
        return 'CORRIENTE'

In [36]:
pagos_corr['DIA_PAGO'] = pagos_corr['FECHA_PAGO'].dt.day
pagos_corr = pagos_corr.rename(columns={'CASA COBRO':'CASA_COBRO'})

In [37]:
pagos_corr['TIPO']=pagos_corr.apply(tipo,axis=1)
cruce_pag_corr=pd.merge(pagos_corr,dias_mora_multa,on=['ID_FACTURA'],how='left')
cruce_pag_corr=cruce_pag_corr[['IDENTIFICACION','ID_FACTURA','VALOR_INFRACCION','FECHA_PAGO','DIA_PAGO','TIPO','DIAS_MORA']]
cruce_pag_corr.head()

,IDENTIFICACION,ID_FACTURA,VALOR_INFRACCION,FECHA_PAGO,DIA_PAGO,TIPO,DIAS_MORA
0,0907964464,6347039,115.8,2019-05-01,1,CORRIENTE,172.0
1,1711723831,5112985,19.3,2019-05-01,1,CORRIENTE,326.0
2,1304396581,5063205,19.3,2019-05-01,1,CORRIENTE,335.0
3,1304396581,5055893,19.3,2019-05-01,1,CORRIENTE,337.0
4,0912125002,6382875,19.7,2019-05-01,1,CORRIENTE,32.0


## MENORES

In [38]:
def identificar_pago(base):
    valor1 = base['VALOR_INFRACCION']
    valor2 = base['VALOR_PAGO_CAPITAL']
    mora = base['DIAS_MORA']
    if mora<=180:
        return valor2
    return valor1

In [39]:
pagos_cuot['TIPO']='CUOTA'
pagos_cuot['DIA_PAGO'] = pagos_cuot['FECHA_REGISTRO_PAGO'].dt.day
pagos_cuot['MONTO_PAGO']=pagos_cuot.apply(identificar_pago,axis=1)
pagos_cuot_resumen=pagos_cuot[['ID','ID_FACTURA_INICIAL','MONTO_PAGO','FECHA_REGISTRO_PAGO','DIA_PAGO','TIPO']]
pagos_cuot_resumen = pagos_cuot_resumen.rename(columns={'ID':'IDENTIFICACION','ID_FACTURA_INICIAL':'ID_FACTURA','MONTO_PAGO':'VALOR_INFRACCION','FECHA_REGISTRO_PAGO':'FECHA_PAGO'})
cruce_pag_cuot=pd.merge(pagos_cuot_resumen,dias_mora_multa,on=['ID_FACTURA'],how='left')
cruce_pag_cuot.head()

,IDENTIFICACION,ID_FACTURA,VALOR_INFRACCION,FECHA_PAGO,DIA_PAGO,TIPO,DIAS_MORA
0,0919709139,5533365,19.30,2019-05-01,1,CUOTA,236.0
1,0919709139,5608800,386.00,2019-05-01,1,CUOTA,218.0
2,0919709139,5973029,112.04,2019-05-01,1,CUOTA,128.0
3,0918099920,6348438,90.64,2019-05-01,1,CUOTA,40.0
4,0919709139,5358553,19.30,2019-05-01,1,CUOTA,273.0


In [40]:
cruce_pag_cuot.columns

Index(['IDENTIFICACION', 'ID_FACTURA', 'VALOR_INFRACCION', 'FECHA_PAGO',
       'DIA_PAGO', 'TIPO', 'DIAS_MORA'],
      dtype='object')

# CONVENIO

In [41]:
def agregar_ceros3(base):
    ced=base['ID']
    if len(ced)==9 or len(ced)==12:
        if ced.isdigit():
            return '0'+ced
    return ced

In [42]:
pagos_conv['TIPO']='CONVENIO'
pagos_conv['DIA_PAGO'] = pagos_conv['FECHA_PAGO'].dt.day

In [43]:
pagos_conv['ID']=pagos_conv.apply(agregar_ceros3,axis=1)

In [44]:
pagos_conv.head()

,ID,ID_CONVENIO,NRO_CUOTA_EMISION,VALOR_PAGAR_CUOTA,FECHA_PAGO,FECHA_VENCIMIENTO,CASA COBRO,TIPO,DIA_PAGO
0,0922469549,306207,6,102.93,2019-05-01,2018-09-19,SISTEMA DE COBRO DEL ECUADOR,CONVENIO,1
1,0922469549,306207,2,102.93,2019-05-01,2018-05-21,SISTEMA DE COBRO DEL ECUADOR,CONVENIO,1
2,0922469549,306207,3,102.93,2019-05-01,2018-06-19,SISTEMA DE COBRO DEL ECUADOR,CONVENIO,1
3,0941443665,350631,1,30.88,2019-05-01,2019-03-18,SISTEMA DE COBRO DEL ECUADOR,CONVENIO,1
4,0941443665,350631,2,30.88,2019-05-01,2019-04-18,SISTEMA DE COBRO DEL ECUADOR,CONVENIO,1


In [45]:
pagos_conv_resumen=pagos_conv[['ID','ID_CONVENIO','VALOR_PAGAR_CUOTA','FECHA_PAGO','DIA_PAGO','TIPO','NRO_CUOTA_EMISION']]
pagos_conv_resumen = pagos_conv_resumen.rename(columns={'ID':'IDENTIFICACION','ID_CONVENIO':'ID_FACTURA','VALOR_PAGAR_CUOTA':'VALOR_INFRACCION','NRO_CUOTA_EMISION':'DIAS_MORA'})

cruce_pag_conv=pd.merge(pagos_conv_resumen,gestion_asesor_cuotas,on=['IDENTIFICACION'],how='left')

In [46]:
cruce_pag_conv.head()

,IDENTIFICACION,ID_FACTURA,VALOR_INFRACCION,FECHA_PAGO,DIA_PAGO,TIPO,DIAS_MORA,COD_AGENTE,FECHA_GESTION,CONTACTO_GESTIONO,ACCION_REALIZADA,RESPUESTA_OBTENIDA,NUMERO_GESTION
0,0922469549,306207,102.93,2019-05-01,1,CONVENIO,6,NaN,NaN,NaN,NaN,NaN,NaN
1,0922469549,306207,102.93,2019-05-01,1,CONVENIO,2,NaN,NaN,NaN,NaN,NaN,NaN
2,0922469549,306207,102.93,2019-05-01,1,CONVENIO,3,NaN,NaN,NaN,NaN,NaN,NaN
3,0941443665,350631,30.88,2019-05-01,1,CONVENIO,1,NaN,NaN,NaN,NaN,NaN,NaN
4,0941443665,350631,30.88,2019-05-01,1,CONVENIO,2,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
#cruce_pag_conv=cruce_pag_conv[['IDENTIFICACION', 'ID_FACTURA', 'VALOR_INFRACCION', 'FECHA_PAGO','DIA_PAGO', 'TIPO', 'DIAS_MORA']]

## UNIR LOS PAGOS

In [48]:
pagos=[cruce_pag_cuot,cruce_pag_corr]
total_pagos=pd.concat(pagos)

In [49]:
total_pagos['TIPO'].value_counts()

CORRIENTE    1019
CUOTA         344
Name: TIPO, dtype: int64

In [50]:
def honorarios(base):
    tipo=base['TIPO']
    mora=base['DIAS_MORA']
    valor=base['VALOR_INFRACCION']
    if tipo=='CONVENIO':
        return valor*0.035
    if tipo=='CUOTA'and mora>180:
        return valor*0.05
    if mora<61:
        return valor*0.02
    if mora<181:
        return valor*0.04
    if mora<361:
        return valor*0.05
    if mora>360:
        return valor*0.06
    return 0.0

In [51]:
total_pagos['HONORARIOS']=total_pagos.apply(honorarios,axis=1)
total_pagos.head()

,IDENTIFICACION,ID_FACTURA,VALOR_INFRACCION,FECHA_PAGO,DIA_PAGO,TIPO,DIAS_MORA,HONORARIOS
0,0919709139,5533365,19.30,2019-05-01,1,CUOTA,236.0,0.9650
1,0919709139,5608800,386.00,2019-05-01,1,CUOTA,218.0,19.3000
2,0919709139,5973029,112.04,2019-05-01,1,CUOTA,128.0,4.4816
3,0918099920,6348438,90.64,2019-05-01,1,CUOTA,40.0,1.8128
4,0919709139,5358553,19.30,2019-05-01,1,CUOTA,273.0,0.9650


In [52]:
total_pagos['TIPO'].value_counts()

CORRIENTE    1019
CUOTA         344
Name: TIPO, dtype: int64

## CRUCE CON GESTIONES

In [53]:
cruce_honorario=pd.merge(total_pagos,total_gestiones,on=['IDENTIFICACION'],how='left')

In [54]:
cruce_honorario.head()

,IDENTIFICACION,ID_FACTURA,VALOR_INFRACCION,FECHA_PAGO,DIA_PAGO,TIPO,DIAS_MORA,HONORARIOS,COD_AGENTE,FECHA_GESTION,CONTACTO_GESTIONO,ACCION_REALIZADA,RESPUESTA_OBTENIDA,NUMERO_GESTION
0,0919709139,5533365,19.30,2019-05-01,1,CUOTA,236.0,0.9650,NaN,NaN,NaN,NaN,NaN,NaN
1,0919709139,5608800,386.00,2019-05-01,1,CUOTA,218.0,19.3000,NaN,NaN,NaN,NaN,NaN,NaN
2,0919709139,5973029,112.04,2019-05-01,1,CUOTA,128.0,4.4816,NaN,NaN,NaN,NaN,NaN,NaN
3,0918099920,6348438,90.64,2019-05-01,1,CUOTA,40.0,1.8128,NaN,NaN,NaN,NaN,NaN,NaN
4,0919709139,5358553,19.30,2019-05-01,1,CUOTA,273.0,0.9650,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
def validar(base):
    dia_pago=base['DIA_PAGO']
    dia_gestion=base['FECHA_GESTION']
    if dia_gestion<=dia_pago:
        return 'SI'
    return 'NO'

In [56]:
cruce_honorario['VALIDO']=cruce_honorario.apply(validar,axis=1)

In [57]:
cruce_pag_conv['HONORARIOS']=cruce_pag_conv.apply(honorarios,axis=1)
cruce_pag_conv['VALIDO']=cruce_pag_conv.apply(validar,axis=1)

In [58]:
pagos_total=[cruce_honorario,cruce_pag_conv]
total_pagos_all=pd.concat(pagos_total)

c:\users\michael\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  


In [59]:
total_pagos_all['TIPO'].value_counts()

CORRIENTE    1019
CONVENIO      486
CUOTA         344
Name: TIPO, dtype: int64

## HONORARIOS VALIDOS

In [60]:
print(total_pagos_all[total_pagos_all['VALIDO']=='SI'].groupby('TIPO').agg({'VALOR_INFRACCION':'sum','HONORARIOS':'sum'}))
print(total_pagos_all[total_pagos_all['VALIDO']=='SI'].groupby('TIPO').agg({'VALOR_INFRACCION':'sum','HONORARIOS':'sum'}).sum())

           VALOR_INFRACCION  HONORARIOS
TIPO                                   
CONVENIO            8706.10    304.7135
CORRIENTE          63404.65   2215.1745
CUOTA              25753.87    961.0088
VALOR_INFRACCION    97864.6200
HONORARIOS           3480.8968
dtype: float64


In [61]:
total_pagos_all[total_pagos_all['VALIDO']=='SI'].groupby('TIPO').agg({'VALOR_INFRACCION':'sum','HONORARIOS':'sum'}).plot.line()

## HONORARIOS TOTALES

In [62]:
print(total_pagos_all.groupby('TIPO').agg({'VALOR_INFRACCION':'sum','HONORARIOS':'sum'}))
print(total_pagos_all.groupby('TIPO').agg({'VALOR_INFRACCION':'sum','HONORARIOS':'sum'}).sum())

           VALOR_INFRACCION  HONORARIOS
TIPO                                   
CONVENIO           31985.27  1119.48445
CORRIENTE          81017.95  2916.22250
CUOTA              30593.03  1148.70090
VALOR_INFRACCION    143596.25000
HONORARIOS            5184.40785
dtype: float64


## HONORARIOS NO VALIDOS

In [63]:
print(total_pagos_all[total_pagos_all['VALIDO']=='NO'].groupby('TIPO').agg({'VALOR_INFRACCION':'sum','HONORARIOS':'sum'}))
print(total_pagos_all[total_pagos_all['VALIDO']=='NO'].groupby('TIPO').agg({'VALOR_INFRACCION':'sum','HONORARIOS':'sum'}).sum())

           VALOR_INFRACCION  HONORARIOS
TIPO                                   
CONVENIO           23279.17   814.77095
CORRIENTE          17613.30   701.04800
CUOTA               4839.16   187.69210
VALOR_INFRACCION    45731.63000
HONORARIOS           1703.51105
dtype: float64


In [64]:
nuevo_archivo='ANALISIS DE PAGOS ANTIGUAS.xlsx'
writer = pd.ExcelWriter(nuevo_archivo, engine='xlsxwriter')
total_pagos_all.to_excel(writer, sheet_name='GESTION TOTAL')
worksheet = writer.sheets['GESTION TOTAL']
writer.save()

In [65]:
nuevo_archivo='GESTION ASESOR.xlsx'
writer = pd.ExcelWriter(nuevo_archivo, engine='xlsxwriter')
gestion_asesor.to_excel(writer, sheet_name='GESTION TOTAL')
worksheet = writer.sheets['GESTION TOTAL']
writer.save()